### Задачи
1. Для понимания эффективности новой фичи нужно посчитать показатели Retention 7 дня  и ARPPU за два периода: до и после внедрения фичи
2. Сформировать выводы по анализу

In [2]:
# импорты
%matplotlib inline
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras

Выгружаем данные из базы за период с 2019-05-01 по 2019-07-01

In [3]:
def getRegisteredUsersCount(): # количество зарегистрированных пользователей  
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall() # список объектов psycopg2.extras.DictCursor
    data = []
    for row in rows:
        data.append(dict(row)) # объект psycopg2.extras.DictCursor превращаеться в словарь
    return data
registered_users_count = pd.DataFrame(getRegisteredUsersCount())

In [4]:
registered_users_count.head(3) # проверка получения

,registration_date,registered_users_count
0,2019-05-01,6516
1,2019-05-02,5309
2,2019-05-03,6266


In [5]:
def getActiveUsersCountWithCohorts(): # когорты и количество пользователей в них
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall() # список объектов psycopg2.extras.DictCursor
    data = []
    for row in rows:
        data.append(dict(row)) # объект psycopg2.extras.DictCursor превращаеться в словарь
    return data 
active_users_count_with_cohorts = pd.DataFrame(getActiveUsersCountWithCohorts())

In [6]:
active_users_count_with_cohorts.head(3) # проверка получения

,activity_date,registration_date,active_users_count
0,2019-05-01,2019-05-01,2590
1,2019-05-02,2019-05-01,2146
2,2019-05-02,2019-05-02,2049


Преобразования и объединение, количества зарегистрированных с количеством пользователей в когортах

In [7]:
# блок преобразований to_datetime
registered_users_count['registration_date'] = pd.to_datetime(registered_users_count['registration_date'])
active_users_count_with_cohorts['activity_date'] = pd.to_datetime(active_users_count_with_cohorts['activity_date'])
active_users_count_with_cohorts['registration_date'] = pd.to_datetime(active_users_count_with_cohorts['registration_date'])

# объединение
retention_table = active_users_count_with_cohorts.merge(registered_users_count,on=['registration_date'],how='left')

# деление когорт по времени жизни 
retention_table['lifetime'] = retention_table['activity_date'] - retention_table['registration_date']

# убираем приставку day оставляем число int
retention_table['lifetime'] = retention_table['lifetime']/np.timedelta64(1,'D')
retention_table['lifetime'] = retention_table['lifetime'].astype(int)

In [8]:
def getRevenue(): # транзакции 
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall() # список объектов psycopg2.extras.DictCursor
    data = []
    for row in rows:
        data.append(dict(row)) # объект psycopg2.extras.DictCursor превращаеться в словарь
    return data
revenue = pd.DataFrame(getRevenue())

In [9]:
revenue.head(3) # проверка получения

,date,registration_date,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,1672,2357.580
1,2019-05-02,2019-05-01,1420,514.679
2,2019-05-02,2019-05-02,1367,1629.140


In [10]:
# преобразование to_datetime
revenue['date'] = pd.to_datetime(revenue['date'])
revenue['registration_date'] = pd.to_datetime(revenue['registration_date'])
# переименования
revenue.rename(columns={'date':'activity_date'},inplace=True)

# создание общей таблицы
retention_table_with_revenue = retention_table.merge(revenue,on=['registration_date','activity_date'],how='left')
# заполнение пустых значений (NaN)
for col in ['revenue','users_count_with_revenue']:
    retention_table_with_revenue[col] = retention_table_with_revenue[col].fillna(0)
    
# немного исправлений 
retention_table_with_revenue['users_count_with_revenue'] = retention_table_with_revenue['users_count_with_revenue'].astype(int)

In [11]:
retention_table_with_revenue.head(3) # конечная таблица для проведения анализа

,activity_date,registration_date,active_users_count,registered_users_count,lifetime,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,2590,6516,0,1672,2357.580
1,2019-05-02,2019-05-01,2146,6516,1,1420,514.679
2,2019-05-02,2019-05-02,2049,5309,0,1367,1629.140


### Retention 7 day

In [12]:
# разделение на 2019-06 и 2019-07 с фильтром по седьмому дню жизни 
retention_7_05 = retention_table_with_revenue[(retention_table_with_revenue['lifetime']==7)
                                              &(retention_table_with_revenue['registration_date']<'2019-06-01')]
retention_7_06 = retention_table_with_revenue[(retention_table_with_revenue['lifetime']==7)
                                              &(retention_table_with_revenue['registration_date']>='2019-06-01')]
# Retention 7 day
retention_05 = retention_7_05['active_users_count'].sum() / retention_7_05['registered_users_count'].sum()
retention_06 = retention_7_06['active_users_count'].sum() / retention_7_06['registered_users_count'].sum()
# вывод
print('Retention седьмого дня равен: для Мая {:.2%} для Июня {:.2%}\nДанный показатель почти не поменялся, с лёгким колебанием в худшую сторону.Пока не ясно хорошо это или плохо, продолжим'.format(retention_05,retention_06))

Retention седьмого дня равен: для Мая 12.93% для Июня 12.56%
Данный показатель почти не поменялся, с лёгким колебанием в худшую сторону.Пока не ясно хорошо это или плохо, продолжим


### ARPPU

In [13]:
# разделение на 2019-06 и 2019-07
retention_revenue_06 = retention_table_with_revenue['revenue'][(retention_table_with_revenue['registration_date']<'2019-06-01')].sum()
retention_revenue_07 = retention_table_with_revenue['revenue'][(retention_table_with_revenue['registration_date']>='2019-06-01')].sum()
# ARPPU
retention_users_06 = retention_table_with_revenue['users_count_with_revenue'][(retention_table_with_revenue['registration_date']<'2019-06-01')].sum()
retention_users_07 = retention_table_with_revenue['users_count_with_revenue'][(retention_table_with_revenue['registration_date']>='2019-06-01')].sum()
# вывод
print('ARPPU Май {:.2%} и Июнь {:.2%}. А тут мы видим что в период после введения фичи средняя прибыль подросла почти на треть.\nТак мы понимаем, что пользователи стали чаще оставлять нам в большие наличности в среднем.'.format(retention_revenue_06/retention_users_06,retention_revenue_07/retention_users_07))

ARPPU Май 30.61% и Июнь 39.32%. А тут мы видим что в период после введения фичи средняя прибыль подросла почти на треть.
Так мы понимаем, что пользователи стали чаще оставлять нам в большие наличности в среднем.


### Выводы

С момента введения фичи Retention не поменялся, но ARPPU вырос почти на 30% что может быть связанно с новшеством, но надо проверить лог самого обновления. Надо дополнительно посчитать сколько платящих пользователей использовали связанный с ним функционал, и только после делать вывод. Но без этого, пока всё очень хорошо и проект успешно ушёл в релиз.